# Introducción:

Trabajaremos con datos bajados de la web https://www.tutiempo.net . Las primeras celdas de código introducen el código necesario para poder obtener los datos que deseamos en forma estructurada (lista de listas o diccionario de diccionarios). 

Los datos que nos interesan es la información del clima horaria de un cierto lugar en un cierto día. 

-----------------------------------------------------------------------------------

# Introduction:

We will work with data downloaded from the web https://www.tutiempo.net. The first code cells introduce the necessary code to obtain the data we want in a structured way (list of lists or dictionary of dictionaries).

The data that interests us is the hourly weather information of a certain place on a certain day.

In [1]:
import requests
from bs4 import BeautifulSoup
import json 

La función `formatear_fecha` recibe una fecha en el formato AAAAMMDD siendo AAAA los años, MM los meses y DD los días y devuelve la misma fecha en formato DD-nombre del mes-AAAA.
La función 'eliminar_unidades' recibe un diccionario del tipo `{'desc': 'Despejado', 'temp': '13°', 'dir': 'Noroeste', 'vel': '7 km/h', 'hum': '88%', 'pres': '1015 hPa'}` para luego modificarlo y devolverlo sin unidades, siguiendo el ejemplo: `{'desc': 'Despejado', 'temp': 13, 'dir': 'Noroeste', 'vel': 7, 'hum': 88, 'pres': 1015}`

Estas funciones me van a ser útiles en la función `registros_dia`

----------------------------------------------------------------------------------------

The `formatear_fecha` function receives a date in the format AAAAMMDD where AAAA is the year, MM is the month and DD is the day, and returns the same date in DD-month name-AAAA format.
The `eliminar_unidades` function receives a dictionary of the type `{'desc': 'Despejado', 'temp': '13°', 'dir': 'Noroeste', 'vel': '7 km/h', 'hum': '88%', 'pres': '1015 hPa'}` and then modify it and return it without units, following the example: `{'desc': 'Despejado', 'temp': 13, 'dir': 'Noroeste', 'vel': 7, 'hum': 88, 'pres': 1015}`

These functions will be useful to me in the `registros_dia` function.


In [2]:
def formatear_fecha(fecha: str) -> str:
    # pre:  fecha es una fecha en el formato 'AAAAMMDD'
    # post: devuelve la fecha  en el foma dia-nombre del mes-año
    assert((type(fecha) == str) and (len(fecha) == 8) and (fecha.isnumeric()))
    meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
      
    return str(int(fecha[6:])) + '-' + meses[int(fecha[4:6])- 1] + '-' + fecha[:4]

def eliminar_unidades(registro: dict):
    # pre:  recibe un diccionario tipo  {'desc': 'Despejado', 'temp': '13°', 'dir': 'Noroeste', 'vel': '7 km/h', 'hum': '88%', 'pres': '1015 hPa'}
    # post: modifica el propio diccionario tipo   {'desc': 'Despejado', 'temp': 13, 'dir': 'Noroeste', 'vel': 7, 'hum': 88, 'pres': 1015}
    assert(type(registro) == dict and ('temp' in registro) and ('vel' in registro) and ('hum' in registro) and ('pres' in registro))
    
    CLAVES_U = ['temp', 'vel', 'hum', 'pres']

    for CLAVE in CLAVES_U:
      i = len(registro[CLAVE])
      if (registro[CLAVE][0] != '-'):
        while ((not(registro[CLAVE].isnumeric())) and (len(registro[CLAVE]) != 0)): 
          registro[CLAVE] = registro[CLAVE][0:i]
          i = i-1
        if (len(registro[CLAVE]) == 0):
          registro[CLAVE] = None
      else:
        registro[CLAVE] = registro[CLAVE][1:]
        while ((not(registro[CLAVE].isnumeric())) and (len(registro[CLAVE]) != 0)):
          registro[CLAVE] = registro[CLAVE][0:i]
          i = i-1
        if ((len(registro[CLAVE])) == 0):
          registro[CLAVE] = None
        else:
          registro[CLAVE] = '-' + registro[CLAVE]
    
    return registro

- Con `requests` podemos bajar una página web.
- Con `BeautifulSoup` extraemos información estructurada de la página.

-----------------------------------------------------------------------------------

- With `requests` we can download a web page.
- With `BeautifulSoup` we extract structured information from the web page.

In [3]:
def registros_dia(estacion: str, fecha: str) -> dict:
  # pre: fecha es una fecha en el formato 'AAAAMMDD' y estacion es una string que indica la estacion meteorologica a analizar
  # pos: devuelve un diccionario de diccionarios con las claves correspondientes 
  #a las horas del dia y dentro de esos diccionarios las claves "hora", "desc", "temp", "dir", "vel", "hum", "pres"
  #que representan los datos climáticos de la estacion a analizar
  
  assert((type(fecha) == str) and (len(fecha) == 8) and (fecha.isnumeric()) and (type(estacion) == str))

  CLAVES = ["hora", "desc", "temp", "dir", "vel", "hum", "pres"]
  nombre_fecha = formatear_fecha(fecha)
  contenido_url = requests.get('https://www.tutiempo.net/registros/'+ estacion +'/' + nombre_fecha + '.html')
  contenido_estructurado = BeautifulSoup(contenido_url.text, 'lxml') 
  tabla_dia = contenido_estructurado.find('table', {'style': 'width: 100%'}) 
  registros = {}
  for fila_tabla in tabla_dia.find_all('tr'):
      celdas_fila = fila_tabla.find_all('td')
      registro = {}
      i = 0
      for celda in celdas_fila:
        if celda.img != None:
          input_tag = celda.img['title']
          registro[CLAVES[i]] = input_tag
          i = i + 1
        registro[CLAVES[i]] = celda.text
        i = i + 1
      if len(registro) != 0:
          hora = registro.pop('hora')
          hh = int(hora[:2])
          registros[hh] = eliminar_unidades(registro)
  return registros

Aquí es donde la magia sucede. La función `clima_anho` recibe un string con la estación meteorológica a analizar, un año en forma de string, el mes en el cual la función va a empezar a analizar y el mes en el que va a finalizar (estos dos últimos en forma de `int`).
Luego va a devolver un archivo de texto del tipo `AAAA_station.txt` siendo `AAAA` el año analizado y `station` la estación meteorológica de donde se obtienen los datos. En este archivo cada renglón hay un diccionario con los datos del clima de las 24 horas de cada día entre los meses indicados anteriormente en la ciudad que corresponde a la estación meteorológica. 

-----------------------------------------------------------------------------------------

Here is where the magic happens. The `clima_anho` function receives a string with the weather station to be analyzed, a year in the form of a string, the month in which the function is going to start analyzing and the month in which it is going to end (these last two in the form of `int`).
Then it will return a text file of the type `YYYY_station.txt` where `YYYY` is the year analyzed and `station` is the weather station from which the data is obtained. In this file, each line contains a dictionary with the climate data for 24 hours of each day between the months indicated above in the city that corresponds to the weather station.


In [5]:
def clima_anho(estacion: str, anho: str, mes_ini: int, mes_fin: int):
  # pre: Recibe una string con la estacion meteorológica a analizar, un año en forma de string, 
  #el mes donde empieza a analizar la funcion y el mes que finaliza de analizar en forma de int
  # pos: Escribe un archivo del tipo 'anho_estacion.txt' en el que en cada renglon hay un diccionario 
  #con los datos del clima de las 24 horas de cada día entre los meses indicados anteriormente en la ciudad que corresponde a la estacion meteorológica
  
  assert((type(anho) == str) and (len(anho) == 4) and (anho.isnumeric()) and (type(mes_ini) == int) and (type(mes_fin) == int) and (mes_ini <= mes_fin) and (type(estacion) == str))

  meses = [31, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
  
  if (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0)):
    meses.insert(1, 29)
  else:
    meses.insert(1, 28)
  
  archivo = anho + '_' + estacion + '.txt'
  res = open(archivo, 'w')
  dicc = {}
  i = mes_ini - 1

  while (i <= (mes_fin - 1)):
    dia = ''
    mes = ''
    if ((i+1) < 10):
      mes = '0' + str(i+1)
    else:
      mes = str(i+1)
    j = 1
    while (j <= (meses[i])):
      if (j < 10):
        dia = '0' + str(j)
      else:
        dia = str(j)
      fecha = anho + mes + dia
      dicc[fecha] = registros_dia(estacion, fecha)
      j = j+1
    i = i+1
  
  
  for claves in dicc:
    mes = ''
    if (mes_fin < 10):
      mes = '0' + str(mes_fin)
    else:
      mes = str(mes_fin)
    if (claves == anho + mes + str(meses[(mes_fin) - 1])):
      res.write('{"' + claves + '": ' + json.dumps(dicc[claves]) + '}')
    else:
      res.write('{"'+ claves + '": ' + json.dumps(dicc[claves]) + '}')
      res.write("\n")
  res.close()

La función `temp_min_max` me va a permitir conocer la temperatura máxima y minima de cada mes del año que estamos analizando de la ciudad a la cual pertenece la estación meteorológica que quiero analizar. Esta función recibe una estación meteorológica, un año a analizar y devuelve una lista de tuplas con la temperatura máxima y minima respectivamente del mes en cuestión.

La función `temp_max` va a recibir una estacion meteorologica y un año en forma de `string`, y un mes inicial, mes final, dia inicial del mes inicial, dia final del mes final en forma de `int`. Esta función me va a servir para calcular el promedio de temperaturas máximas entre las fechas previamente provistas. La función devolverá un entero con dicho número.

La función `dir_viento` recibe una estacion meteorologica y un año en forma de `string`, también recibe un mes inicial, mes final, dia inicial del mes inicial, dia final del mes final en forma de `int`. Esta función me va a servir para conocer la/s dirección/es  predominante/s del viento durante las fechas previamente proporcionadas (es decir la moda).

-----------------------------------------------------------------------------------------

The `temp_min_max` function will allow me to know the maximum and minimum temperature of each month of the year that we are analyzing of the city to which the weather station that I want to analyze belongs to. This function receives a weather station, a year to analyze and returns a list of tuples with the maximum and minimum temperature, respectively, of the previously provided month.

The `temp_max` function will receive a weather station and a year as a string, also will receive a start month, end month, start day of the start month, end day of the end month as an int. This function will help me calculate the average maximum temperatures between the dates previously provided. The function will return an integer with that number.

The `wind_dir` function receives a weather station and a year in the form of a `string`, it also receives a start month, end month, start day of the start month, end day of the end month in the form of `int`. This function will help me to know the predominant direction(s) of the wind during the dates previously provided.

In [6]:
def temp_min_max(estacion, anho):
  # pre: Recibe una estacion meteorológica y un año, ambos en forma de string
  # pos: Devuelve una lista de tuplas con la temperatura maxima y minima de cada mes respectivamente del archivo 'anho_estacion.txt'
  
  assert((type(estacion) == str) and (type(anho) == str) and (len(anho) == 4))

  bis = (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0))
  DIAS_MESES_ANTERIORES = [31, 59 + bis, 90 + bis, 120 + bis, 151 + bis, 181 + bis, 212 + bis, 243 + bis, 273 + bis, 304 + bis, 334 + bis, 365 + bis]

  arch = open("./"+ anho + '_' + estacion + ".txt", "r")
  arch_lines = arch.readlines()
  dicts = []
  for lines in arch_lines:
    dicts.append(json.loads(lines))
  arch.close()

  max_y_mins = []
  ind = 0
  for can_dias in DIAS_MESES_ANTERIORES:
    temp_min = float("inf")
    temp_max = -274
    while (ind < can_dias):
      fecha = list(dicts[ind].keys())[0]
      for h in range(24):
        if (((str(h)) in (dicts[ind][fecha])) and (type(dicts[ind][fecha][str(h)]['temp']) == str)):
          temp = int(dicts[ind][fecha][str(h)]['temp'])
          if (temp > temp_max):
            temp_max = temp
          if (temp < temp_min):
            temp_min = temp                      
      ind = ind + 1
    max_y_mins.append((temp_max, temp_min))
  
  return max_y_mins

def temp_max(estacion, anho, mes_ini, mes_fin, dia_mes_ini, dia_mes_fin):
  # pre: Recibe una estacion meteorologica y un año en forma de string, y un mes inicial, 
  # mes final, dia inicial del mes inicial, dia final del mes final en forma de int
  # pos: Devuelve un int que representa el promedio de temperaturas maximas de 
  # cada día entre dia_mes_ini/mes_ini/anho y dia_mes_fin/mes_fin/anho

  assert((type(estacion) == str) and (type(anho) == str) and (type(mes_ini) == int) and (type(mes_fin) == int) and (type(dia_mes_ini) == int) and (type(dia_mes_fin) == int) and (len(anho) == 4) and (mes_ini <= mes_fin))



  DIAS_MESES_ANTERIORES = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]

  dias_mes_ini = dia_mes_ini + DIAS_MESES_ANTERIORES[mes_ini-1]
  if (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0)) and mes_ini >= 3:
    dias_mes_ini = dias_mes_ini + 1

  dias_mes_fin = dia_mes_fin + DIAS_MESES_ANTERIORES[mes_fin-1]
  if (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0)) and mes_ini >= 3:
    dias_mes_fin = dias_mes_fin + 1

  arch = open("./"+ anho + '_' + estacion +".txt", "r")
  arch_lines = arch.readlines()
  dicts = []
  for lines in arch_lines:
    dicts.append(json.loads(lines))
  arch.close()

  maximas = []
  ind = dias_mes_ini -1
  while (ind <= (dias_mes_fin-1)):
    temp_max = -274
    fecha = list(dicts[ind].keys())[0]
    for h in range(24):
      if (((str(h)) in (dicts[ind][fecha])) and (type(dicts[ind][fecha][str(h)]['temp']) == str)):
        temp = int(dicts[ind][fecha][str(h)]['temp'])
        if (temp > temp_max):
          temp_max = temp
    maximas.append(temp_max)
    ind = ind + 1  
  res = 0
  for temp in maximas:
    res = res + temp  
  res = res / (len(maximas))
  
  return res



def dir_viento(estacion, anho, mes_ini, mes_fin, dia_mes_ini, dia_mes_fin):
  # pre: Recibe una estacion meteorologica y un año en forma de string, y un mes inicial, 
  # mes final, dia inicial del mes inicial, dia final del mes final en forma de int
  # pos: Devuelve una lista con la/s direccion/es del viento predominante/s entre dia_mes_ini/mes_ini/anho y dia_mes_fin/mes_fin/anho
  
  assert((type(estacion) == str) and (type(anho) == str) and (type(mes_ini) == int) and (type(mes_fin) == int) and (type(dia_mes_ini) == int) and (type(dia_mes_fin) == int) and (len(anho) == 4) and (mes_ini <= mes_fin))


  DIAS_MESES_ANTERIORES = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]

  dias_mes_ini = dia_mes_ini + DIAS_MESES_ANTERIORES[mes_ini-1]
  if (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0)) and mes_ini >= 3:
    dias_mes_ini = dias_mes_ini + 1

  dias_mes_fin = dia_mes_fin + DIAS_MESES_ANTERIORES[mes_fin-1]
  if (((int(anho)) % 4 == 0 and (not ((int(anho)) % 100 == 0))) or ((int(anho)) % 100 == 0 and (int(anho)) % 400 == 0)) and mes_ini >= 3:
    dias_mes_fin = dias_mes_fin + 1

  arch = open("./"+ anho + '_' + estacion +".txt", "r")
  arch_lines = arch.readlines()
  dicts = []
  for lines in arch_lines:
    dicts.append(json.loads(lines))
  arch.close()

  dir_vien = []
  ind = dias_mes_ini -1
  while (ind <= (dias_mes_fin-1)):
    temp_max = -274
    fecha = list(dicts[ind].keys())[0]
    for h in range(24):
      if (((str(h)) in (dicts[ind][fecha])) and (type(dicts[ind][fecha][str(h)]['dir']) == str)):
        dir_vien.append(dicts[ind][fecha][str(h)]['dir'])        
    ind = ind + 1  
  
  res = []
  count_dires = 0
  for dires in dir_vien:
    if ((dir_vien.count(dires)) > count_dires):
      count_dires = dir_vien.count(dires)
  for dires in dir_vien:
    if (((dir_vien.count(dires)) == count_dires) and ((res.count(dires)) < 1)):
      res.append(dires)
  
  return res

Por favor ingresá las fechas a analizar.

--------------------------------------------------------------------------------------

Please enter the dates to analyze.

In [7]:
print("Enter the name of the weather station to analyze: ")

estacion = input()

print("Enter the year to analyze: ")

anho = input()

print("Enter the month from where you will start to analyze the weather: ")

primer_mes = int(input())

print("Place the day of the month previously placed: ")

dia_primer_mes = int(input())

print("Enter the month where you will finish analyzing the weather: ")

ultimo_mes = int(input())

print("Place the day of the month previously placed: ")

dia_ultimo_mes = int(input())

print("The dates you entered are: " + str(dia_primer_mes) + "/" + str(primer_mes) + "/" + str(anho) + " and " + str(dia_ultimo_mes) + "/" + str(ultimo_mes) + "/" + str(anho))

Enter the name of the weather station to analyze: 
Enter the year to analyze: 
Enter the month from where you will start to analyze the weather: 
Place the day of the month previously placed: 
Enter the month where you will finish analyzing the weather: 
Place the day of the month previously placed: 
The dates you entered are: 1/1/2018 and 31/12/2018


Aquí se ejecutan las funciones mencionadas anteriormente y podemos conocer las temperaturas máximas y minimas de cada mes entre las fechas que fueron provistas como así también el promedio de temperaturas máximas y la dirección/es predominante/s del viento.

---------------------------------------------------------------------------------------

Here the functions mentioned above are executed and we can know the maximum and minimum temperatures of each month between the dates that were provided as well as the average maximum temperatures and the predominant wind direction(s).

In [8]:
clima_anho(estacion, anho, primer_mes, ultimo_mes)

print("The maximum and minimum temperatures respectively for each month of " + anho + " in the weather station " + estacion + " are " + str(temp_min_max(estacion, anho)))
print("The average daily maximum temperatures from " + str(dia_primer_mes) + "/" + str(primer_mes) + "/" + anho + " until " + str(dia_ultimo_mes) + "/" + str(ultimo_mes) + "/" + anho + " in the weather station " + estacion + " is: " + str(temp_max(estacion, anho, primer_mes, ultimo_mes, dia_primer_mes, dia_ultimo_mes)) + '°C' )
print("The prevailing wind direction/s from " + str(dia_primer_mes) + "/" + str(primer_mes) + "/" + anho + " until " + str(dia_ultimo_mes) + "/" + str(ultimo_mes) + "/" + anho + " in the weather station " + estacion + " is/are: " + str(dir_viento(estacion, anho, primer_mes, ultimo_mes, dia_primer_mes, dia_ultimo_mes)))

The maximum and minimum temperatures respectively for each month of 2018 in the weather station sazs are [(32, 0), (33, 0), (28, -2), (24, -4), (17, -6), (14, -11), (9, -9), (14, -7), (18, -6), (20, -5), (25, -3), (29, -1)]
The average daily maximum temperatures from 1/1/2018 until 31/12/2018 in the weather station sazs is: 14.10958904109589°C
The prevailing wind direction/s from 1/1/2018 until 31/12/2018 in the weather station sazs is/are: ['Noroeste']
